데이콘 Basic 전화 해지 여부 분류 AI 경진대회

https://dacon.io/competitions/official/236075/

<pre>
30200개의 데이터
ID : 샘플 별 고유 ID
가입일 : 서비스에 가입한 일수
음성사서함이용 : 음성사서함 이용 건수
주간통화시간 : 8시 ~ 16시까지의 통화 시간
주간통화횟수 : 8시 ~ 16시까지의 통화 횟수
주간통화요금 : 8시 ~ 16시까지의 통화 요금
저녁통화시간 : 16시 ~ 0시까지의 통화 시간
저녁통화횟수 : 16시 ~ 0시까지의 통화 횟수
저녁통화요금 : 16시 ~ 0시까지의 통화 요금
밤통화시간 : 0시 ~ 8시까지의 통화 시간
밤통화횟수 : 0시 ~ 8시까지의 통화 횟수
밤통화요금 : 0시 ~ 8시까지의 통화 요금
상담전화건수 : 고객센터에 전화를 건 횟수
전화해지여부 : 0(서비스 유지)/ 1(서비스 해지)

In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import re
from datetime import datetime, date, time, timedelta
from dateutil.relativedelta import relativedelta

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
import seaborn as sns
# plt.rcParams['figure.figsize'] = [6.4, 4.8]

from imblearn.under_sampling import RandomUnderSampler, TomekLinks
from imblearn.over_sampling import SMOTE, RandomOverSampler, ADASYN
from imblearn.combine import SMOTEENN, SMOTETomek

from sklearn.datasets import load_iris, load_wine, load_diabetes
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler, Binarizer, label_binarize
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, VotingRegressor, RandomForestRegressor, ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.metrics import precision_score, precision_recall_curve#, plot_precision_recall_curve
from sklearn.metrics import auc, roc_auc_score, roc_curve#, plot_roc_curve
from sklearn.metrics import mean_squared_error, mean_squared_log_error, r2_score
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier

import missingno as msno

from pycaret.classification import *

sns.set()
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 데이터로드 target=전화해지여부

In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
score_df = pd.read_csv('score_df.csv')
train = train.drop('ID', axis=1)
test = test.drop('ID', axis=1)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30200 entries, 0 to 30199
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   가입일      30200 non-null  int64  
 1   음성사서함이용  30200 non-null  int64  
 2   주간통화시간   30200 non-null  float64
 3   주간통화횟수   30200 non-null  int64  
 4   주간통화요금   30200 non-null  float64
 5   저녁통화시간   30200 non-null  float64
 6   저녁통화횟수   30200 non-null  int64  
 7   저녁통화요금   30200 non-null  float64
 8   밤통화시간    30200 non-null  float64
 9   밤통화횟수    30200 non-null  int64  
 10  밤통화요금    30200 non-null  float64
 11  상담전화건수   30200 non-null  int64  
 12  전화해지여부   30200 non-null  int64  
dtypes: float64(6), int64(7)
memory usage: 3.0 MB


In [3]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12943 entries, 0 to 12942
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   가입일      12943 non-null  int64  
 1   음성사서함이용  12943 non-null  int64  
 2   주간통화시간   12943 non-null  float64
 3   주간통화횟수   12943 non-null  int64  
 4   주간통화요금   12943 non-null  float64
 5   저녁통화시간   12943 non-null  float64
 6   저녁통화횟수   12943 non-null  int64  
 7   저녁통화요금   12943 non-null  float64
 8   밤통화시간    12943 non-null  float64
 9   밤통화횟수    12943 non-null  int64  
 10  밤통화요금    12943 non-null  float64
 11  상담전화건수   12943 non-null  int64  
dtypes: float64(6), int64(6)
memory usage: 1.2 MB


In [4]:
train.head()

,가입일,음성사서함이용,주간통화시간,주간통화횟수,주간통화요금,저녁통화시간,저녁통화횟수,저녁통화요금,밤통화시간,밤통화횟수,밤통화요금,상담전화건수,전화해지여부
0,329,0,99.2,93,27.3,268.8,68,28.92,262.9,328,32.89,2,0
1,2,80,323.9,323,83.7,269.4,326,32.09,322.8,209,32.32,2,0
2,93,28,282.4,323,34.2,207.0,322,32.82,280.8,328,8.28,0,0
3,223,1,221.4,223,25.1,233.0,61,23.90,203.8,234,9.36,0,0
4,222,0,96.3,222,28.7,223.9,69,28.08,263.1,223,2.80,8,0


In [5]:
train.columns

Index(['가입일', '음성사서함이용', '주간통화시간', '주간통화횟수', '주간통화요금', '저녁통화시간', '저녁통화횟수',
       '저녁통화요금', '밤통화시간', '밤통화횟수', '밤통화요금', '상담전화건수', '전화해지여부'],
      dtype='object')

In [6]:
df0 = train[train['전화해지여부']==0]
df1 = train[train['전화해지여부']==1]
stat_df = pd.concat([df0.describe().T,df1.describe().T], axis=1)
stat_df

,count,mean,std,min,25%,50%,75%,max,count,mean,std,min,25%,50%,75%,max
가입일,26882.0,159.441857,124.193729,1.00,63.00,116.00,228.00,2212.00,3318.0,162.088608,120.112890,1.00,67.00,135.00,229.000,2012.00
음성사서함이용,26882.0,12.587159,25.566918,0.00,0.00,1.00,23.00,1112.00,3318.0,7.587101,18.348552,0.00,0.00,1.00,1.000,117.00
주간통화시간,26882.0,249.674619,84.830161,0.00,222.50,233.60,289.40,481.20,3318.0,257.764708,80.513591,0.10,224.20,250.25,290.800,481.90
주간통화횟수,26882.0,158.784131,101.747847,0.00,88.00,115.00,223.00,489.00,3318.0,162.348403,100.104352,0.00,88.00,143.00,223.000,489.00
주간통화요금,26882.0,41.669660,23.536478,0.00,24.50,31.20,46.20,118.60,3318.0,36.406841,19.427776,0.10,24.00,29.80,40.575,117.80
저녁통화시간,26882.0,263.251451,74.741785,0.10,223.70,242.95,290.40,481.60,3318.0,263.212899,68.344831,21.10,223.90,244.10,289.900,481.40
저녁통화횟수,26882.0,158.628078,102.359429,0.00,87.00,112.00,223.00,489.00,3318.0,157.230561,100.908386,20.00,87.00,111.00,223.000,489.00
저녁통화요금,26882.0,25.389364,8.223539,0.00,22.22,23.30,28.82,49.98,3318.0,25.433927,7.490529,2.29,22.22,23.32,28.880,49.89
밤통화시간,26882.0,263.512730,74.529181,20.80,223.70,242.70,290.50,481.80,3318.0,262.835142,67.395499,22.60,223.70,244.30,290.100,481.00
밤통화횟수,26882.0,157.158954,101.785060,20.00,87.00,107.00,222.00,490.00,3318.0,156.143761,101.216735,20.00,83.00,110.00,223.000,488.00


In [7]:
score_df.sort_values('f1', ascending=True)

NameError: name 'score_df' is not defined

# EDA

In [ ]:
plt.figure(figsize=(14,10))
for i, col in enumerate(['가입일', '음성사서함이용', '주간통화시간', '주간통화횟수', '주간통화요금', '저녁통화시간', '저녁통화횟수',
       '저녁통화요금', '밤통화시간', '밤통화횟수', '밤통화요금', '상담전화건수']):
    plt.subplot(3,4,i+1)
    plt.title(col)
    sns.boxplot(data=train, x='전화해지여부', y=col)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14,10))
for i, col in enumerate(['가입일', '음성사서함이용', '주간통화시간', '주간통화횟수', '주간통화요금', '저녁통화시간', '저녁통화횟수',
       '저녁통화요금', '밤통화시간', '밤통화횟수', '밤통화요금', '상담전화건수']):
    plt.subplot(3,4,i+1)
    plt.title(col)
    sns.violinplot(data=train, x='전화해지여부', y=col)
plt.tight_layout()
plt.show()

# 기준점수 0.58533

In [ ]:
def get_f1_score(train, model=DecisionTreeClassifier(random_state=0), cv=5):
    model = ExtraTreesClassifier(random_state=0)
    y = train['전화해지여부']
    X = train.drop('전화해지여부', axis=1)
    f1 = np.mean(cross_val_score(model, X, y, scoring='f1', cv=cv))
    
    return np.round(f1, 5)

get_f1_score(train)

# 6표준편차로 이상치 처리

In [ ]:
def six_sigma(df):
    if '전화해지여부' in df.columns:
        X = df.drop('전화해지여부', axis=1)
        y = df['전화해지여부']
        df_type = 'train'
    else:
        X = df
        df_type = 'test'
    
    for col in X.columns:
        if col!='상담전화건수':
            mean_ = X[col].mean()
            std_ = X[col].std()
            idx_under = X[(X[col]<mean_-std_*3)].index
            X.loc[idx_under, col] = mean_-std_*3
            idx_over = X[(X[col]>mean_+std_*3)].index
            X.loc[idx_over, col] = mean_+std_*3
        
    if df_type == 'train':
        df = pd.concat([X,y], axis=1)
    elif df_type == 'test':
        df = X
        
    return df

train_six_sigma = six_sigma(train)
# get_f1_score(train_six_sigma)

## 점수 0.57517

In [ ]:
plt.figure(figsize=(14,10))
for i, col in enumerate(['가입일', '음성사서함이용', '주간통화시간', '주간통화횟수', '주간통화요금', '저녁통화시간', '저녁통화횟수',
       '저녁통화요금', '밤통화시간', '밤통화횟수', '밤통화요금', '상담전화건수']):
    plt.subplot(3,4,i+1)
    plt.title(col)
    sns.violinplot(data=train_six_sigma, x='전화해지여부', y=col)
plt.tight_layout()
plt.show()

# 기준비율 10.99%

In [ ]:
def 비율(train, count=0):
    (해지0, 해지1) = train['전화해지여부'].value_counts().sort_index().values
    if count==1:
        print(해지0, 해지1)
    return print(f'{np.round(해지1/(해지0+해지1)*100,2)}%')

비율(train)

# 가입일

In [ ]:
sns.violinplot(data=train_six_sigma, x='전화해지여부', y='가입일')
plt.show()

In [ ]:
sns.histplot(data=train_six_sigma, x='가입일', hue='전화해지여부', kde=True)
plt.show()

##  가입일 -> 가입시즌

In [ ]:
def make_가입시즌(df):
    # dfcopy = six_sigma(df)
    dfcopy = df.copy()
    dfcopy['가입시즌'] = dfcopy['가입일']//200
    return dfcopy

가입시즌_df = make_가입시즌(train)
가입시즌_df.head(1)

In [ ]:
for 시즌 in [0,1,2,4,10]:
    비율(가입시즌_df[가입시즌_df['가입시즌']==시즌][['전화해지여부']])

In [ ]:
가입시즌_df[['가입시즌', '전화해지여부']].value_counts().sort_index()

## 점수 0.5876

In [ ]:
get_f1_score(가입시즌_df)

# <del>음성사서함</del>

In [ ]:
six_sigma(train)['음성사서함이용'].hist()
plt.show()

In [ ]:
sns.histplot(data=six_sigma(train), x='음성사서함이용', kde=True)
plt.show()

In [ ]:
비율(six_sigma(train)[six_sigma(train)['음성사서함이용']==0], count=1)

In [ ]:
비율(six_sigma(train)[six_sigma(train)['음성사서함이용']!=0], count=1)

## 음성사서함 -> 음성사서함유무

In [ ]:
def make_음성사서함유무(df):
    dfcopy = df.copy()
    dfcopy['음성사서함유무'] = 0
    dfcopy['음성사서함유무'].loc[dfcopy[dfcopy['음성사서함이용']!=0].index] = 1
    return dfcopy

음성사서함유무_df = make_음성사서함유무(train)
음성사서함유무_df.head(2)

In [ ]:
get_f1_score(train_six_sigma)

In [ ]:
비율(음성사서함유무_df)

## 점수 0.4831

In [ ]:
get_f1_score(음성사서함유무_df)

# <del>3종시간</del>

In [ ]:
시간리스트 = ['주간통화시간', '저녁통화시간', '밤통화시간']
plt.figure(figsize=(14,14))
for i, 시간 in enumerate(시간리스트):
    plt.subplot(3,1,i+1)
    sns.histplot(data=six_sigma(train), x=시간, kde=True)
    plt.xlim(-10,510)
plt.show()

In [ ]:
def 시간3분할(df):
    dfcopy = df.copy()
    for 시간 in ['주간통화시간', '저녁통화시간', '밤통화시간']:
        dfcopy[f'{시간}구간'] = 0
        dfcopy[f'{시간}구간'].loc[dfcopy[dfcopy[시간]<150].index] = 1
        dfcopy[f'{시간}구간'].loc[dfcopy[(dfcopy[시간]>=150)&(dfcopy[시간]<380)].index] = 2
        dfcopy[f'{시간}구간'].loc[dfcopy[dfcopy[시간]>=380].index] = 3
        
    return dfcopy

시간3분할_df = 시간3분할(train)
시간3분할_df.head()

In [ ]:
for 시간 in ['주간통화시간구간', '저녁통화시간구간', '밤통화시간구간']:
    for 구간 in [1,2,3]:
        비율(시간3분할_df[시간3분할_df[시간]==구간])
    print('-----')

## 점수보기 0.58316

In [ ]:
get_f1_score(시간3분할_df)

# 3종요금

In [ ]:
요금리스트 = ['주간통화요금', '저녁통화요금', '밤통화요금']
plt.figure(figsize=(14,14))
for i, 요금 in enumerate(요금리스트):
    plt.subplot(3,1,i+1)
    sns.histplot(data=six_sigma(train), x=요금, kde=True)
    plt.xlim(-10,110)
plt.show()

In [ ]:
six_sigma(train)['밤통화요금'].max()

## 요금3분할

In [ ]:
def 요금3분할(df):
    dfcopy = df.copy()
    for 요금 in ['주간통화요금', '저녁통화요금', '밤통화요금']:
        dfcopy[f'{요금}구간'] = 0
        dfcopy[f'{요금}구간'].loc[dfcopy[dfcopy[요금]<40.4].index] = 1
        dfcopy[f'{요금}구간'].loc[dfcopy[(dfcopy[요금]>=40.4)&(dfcopy[요금]<70)].index] = 2
        dfcopy[f'{요금}구간'].loc[dfcopy[dfcopy[요금]>=70].index] = 3
        
    return dfcopy

요금3분할_df = 요금3분할(train)
요금3분할_df.head()

In [ ]:
비율(요금3분할_df[요금3분할_df['주간통화요금구간']==1])
비율(요금3분할_df[요금3분할_df['주간통화요금구간']==2])
비율(요금3분할_df[요금3분할_df['주간통화요금구간']==3])
비율(요금3분할_df[요금3분할_df['저녁통화요금구간']==1])
비율(요금3분할_df[요금3분할_df['저녁통화요금구간']==2])
비율(요금3분할_df[요금3분할_df['밤통화요금구간']==1])

## 점수보기 0.58703

In [ ]:
get_f1_score(요금3분할_df)

# 상담전화건수

In [ ]:
train['상담전화건수'].hist()

In [ ]:
sns.countplot(data=train, x='상담전화건수')
plt.show()

In [ ]:
for i in np.sort(train['상담전화건수'].unique()):
    print(i, end=' ')
    비율(train[train['상담전화건수']==i])

# 분석끝

In [ ]:
get_f1_score(시간3분할(요금3분할(make_가입시즌(train))))

In [ ]:
get_f1_score(six_sigma(시간3분할(요금3분할(make_가입시즌(train)))))

In [ ]:
get_f1_score(make_음성사서함유무(시간3분할(요금3분할(make_가입시즌(train)))))

# 고려대상

In [ ]:
train['총요금'] = train['주간통화요금']+train['저녁통화요금']+train['밤통화요금']
train

# 시간대비요금 바보와 얌체

In [ ]:
plt.figure(figsize=(14,14))

for i, col_tuplt in enumerate([('주간통화시간', '주간통화요금'), ('저녁통화시간', '저녁통화요금'), ('밤통화시간', '밤통화요금')]):
    plt.subplot(3,3,i+1)
    sns.scatterplot(data=train[train['전화해지여부']==0], x=col_tuplt[0], y=col_tuplt[1])
    plt.ylim([-10, 70])
    
for i, col_tuplt in enumerate([('주간통화시간', '주간통화요금'), ('저녁통화시간', '저녁통화요금'), ('밤통화시간', '밤통화요금')]):
    plt.subplot(3,3,i+4)
    sns.scatterplot(data=train, x=col_tuplt[0], y=col_tuplt[1], hue='전화해지여부')
    plt.ylim([-10, 70])
    
for i, col_tuplt in enumerate([('주간통화시간', '주간통화요금'), ('저녁통화시간', '저녁통화요금'), ('밤통화시간', '밤통화요금')]):
    plt.subplot(3,3,i+7)
    sns.scatterplot(data=train[train['전화해지여부']==1], x=col_tuplt[0], y=col_tuplt[1], color='Orange')
    plt.ylim([-10, 70])
    
plt.show()

In [ ]:
plt.figure(figsize=(14,4))
target_feature = [('주간통화시간', '주간통화요금', 0.173+0.01), ('저녁통화시간', '저녁통화요금', 0.085-0.005), ('밤통화시간', '밤통화요금', 0.045)]
for i, col_tuplt in enumerate(target_feature):
    plt.subplot(1,3,i+1)
    sns.scatterplot(data=train_six_sigma, x=col_tuplt[0], y=col_tuplt[1], hue='전화해지여부')
    plt.plot(train_six_sigma[col_tuplt[0]], train_six_sigma[col_tuplt[0]]*col_tuplt[2], 'r-')
    plt.ylim([-10, 130])
    
plt.show()

In [ ]:
plt.figure(figsize=(14,4))
target_feature = [('주간통화시간', '주간통화요금', 0.173+0.01), ('저녁통화시간', '저녁통화요금', 0.085-0.005), ('밤통화시간', '밤통화요금', 0.045)]
for i, col_tuplt in enumerate(target_feature):
    plt.subplot(1,3,i+1)
    sns.scatterplot(data=train, x=col_tuplt[0], y=col_tuplt[1], hue='전화해지여부')
    plt.plot(train[col_tuplt[0]], train[col_tuplt[0]]*col_tuplt[2], 'r-')
    plt.ylim([-10, 70])
    
plt.show()

In [ ]:
def 바보얌체(df):
    dfcopy = df.copy()
    
    dfcopy['주간바보얌체'] = 0
    dfcopy['주간바보얌체'].loc[dfcopy[dfcopy['주간통화요금']/dfcopy['주간통화시간']>0.173+0.01].index] = -1
    dfcopy['주간바보얌체'].loc[dfcopy[(dfcopy['주간통화요금']/dfcopy['주간통화시간']<=0.173+0.01)&(dfcopy['주간통화요금']/dfcopy['주간통화시간']>=0.173-0.01)].index] = 0
    dfcopy['주간바보얌체'].loc[dfcopy[dfcopy['주간통화요금']/dfcopy['주간통화시간']<0.173-0.01].index] = 1
    
    dfcopy['저녁바보얌체'] = 0
    dfcopy['저녁바보얌체'].loc[dfcopy[dfcopy['저녁통화요금']/dfcopy['저녁통화시간']>0.085+0.005].index] = -1
    dfcopy['저녁바보얌체'].loc[dfcopy[(dfcopy['저녁통화요금']/dfcopy['저녁통화시간']<=0.085+0.005)&(dfcopy['저녁통화요금']/dfcopy['저녁통화시간']>=0.085-0.005)].index] = 0
    dfcopy['저녁바보얌체'].loc[dfcopy[dfcopy['저녁통화요금']/dfcopy['저녁통화시간']<0.085-0.005].index] = 1
    
    dfcopy['밤바보얌체'] = 0
    dfcopy['밤바보얌체'].loc[dfcopy[dfcopy['밤통화요금']/dfcopy['밤통화시간']>0.045+0.003].index] = -1
    dfcopy['밤바보얌체'].loc[dfcopy[(dfcopy['밤통화요금']/dfcopy['밤통화시간']<=0.045+0.003)&(dfcopy['밤통화요금']/dfcopy['밤통화시간']>=0.045-0.003)].index] = 0
    dfcopy['밤바보얌체'].loc[dfcopy[dfcopy['밤통화요금']/dfcopy['밤통화시간']<0.045-0.003].index] = 1
    
    dfcopy['바보얌체점수'] = dfcopy['주간바보얌체']+dfcopy['저녁바보얌체']+dfcopy['밤바보얌체']
    
    return dfcopy

바보얌체_df = 바보얌체(train)
바보얌체_df.head()

In [ ]:
for n in [-3,-2,-1,0,1,2,3]:
    print(n, end=' ')
    비율(바보얌체_df[바보얌체_df['바보얌체점수']==n])

In [ ]:
get_f1_score(바보얌체(시간3분할(요금3분할(make_가입시즌(train)))))

In [ ]:
get_f1_score(six_sigma(바보얌체(시간3분할(요금3분할(make_가입시즌(train))))))

In [ ]:
get_f1_score(make_음성사서함유무(바보얌체(시간3분할(요금3분할(make_가입시즌(train))))))

# 스케일링+겟더미

In [ ]:
바보얌체(시간3분할(요금3분할(make_가입시즌(train)))).columns

In [ ]:
def scaling_get_dummies(df, sc=0, log=0):
    if '전화해지여부' in df.columns:
        X = df.drop('전화해지여부', axis=1)
        y = df['전화해지여부']
        df_type = 'train'
    else:
        X = df
        df_type = 'test'
        
    이산형컬럼명 = ['가입시즌', '주간통화시간구간', '저녁통화시간구간', '밤통화시간구간', '주간바보얌체', '저녁바보얌체', '밤바보얌체', '바보얌체점수']
    이산형data = X[이산형컬럼명]
    
    명목형data = pd.get_dummies('상담전화'+X['상담전화건수'].astype('str')+'건')#.drop(['상담전화11건'], axis=1)
    
    연속형컬럼명 = ['가입일', '음성사서함이용','주간통화시간', '주간통화횟수', '주간통화요금', '저녁통화시간', '저녁통화횟수', '저녁통화요금', '밤통화시간', '밤통화횟수', '밤통화요금']
    연속형data = X[연속형컬럼명]
    
    if (log==1)|(log==True):
        연속형data = np.log1p(연속형data)
    scale_list = [StandardScaler(), MinMaxScaler(), RobustScaler()]
    연속형data = pd.DataFrame(scale_list[sc].fit_transform(연속형data), columns=연속형data.columns.values)
    
    X_scale = pd.concat([pd.concat([연속형data, 명목형data], axis=1), 이산형data], axis=1)
    
    
    
    if df_type == 'train':
        df = pd.concat([X_scale,y], axis=1)
    elif df_type == 'test':
        df = X_scale
    return df

scaling_get_dummies(바보얌체(시간3분할(요금3분할(make_가입시즌(test))))).head()

## 점수

0 1 2 [StandardScaler(), MinMaxScaler(), RobustScaler()]

In [ ]:
train_우리피쳐 = 바보얌체(시간3분할(요금3분할(make_가입시즌(train))))
for sc in [0,1,2]:
    for log in [0,1]:
        print(get_f1_score(scaling_get_dummies(train_우리피쳐, sc=sc, log=log)))

In [ ]:
train = scaling_get_dummies(train_우리피쳐, sc=2)
y = train['전화해지여부']
X = train.drop(['전화해지여부'], axis=1)

# <del>Over Sampling</del>

In [ ]:
def trin_over_sampling(train, stype=1, chart=0):
    y = train['전화해지여부']
    X = train.drop(['전화해지여부'], axis=1)
    
    if stype==0:
        X_, y_ = SMOTE(random_state=11).fit_resample(X, y)
    elif stype==1:
        X_, y_ = RandomOverSampler(random_state=0).fit_resample(X, y)
    
    if chart==1:
        df = pd.concat([X_, y_], axis=1)
        plt.figure(figsize=(14,14))
        for i, col_tuplt in enumerate([('주간통화시간', '주간통화요금'), ('저녁통화시간', '저녁통화요금'), ('밤통화시간', '밤통화요금')]):
            plt.subplot(3,3,i+1)
            sns.scatterplot(data=df[df['전화해지여부']==0], x=col_tuplt[0], y=col_tuplt[1])
            plt.ylim([-10, 70])

        for i, col_tuplt in enumerate([('주간통화시간', '주간통화요금'), ('저녁통화시간', '저녁통화요금'), ('밤통화시간', '밤통화요금')]):
            plt.subplot(3,3,i+4)
            sns.scatterplot(data=df, x=col_tuplt[0], y=col_tuplt[1], hue='전화해지여부')
            plt.ylim([-10, 70])

        for i, col_tuplt in enumerate([('주간통화시간', '주간통화요금'), ('저녁통화시간', '저녁통화요금'), ('밤통화시간', '밤통화요금')]):
            plt.subplot(3,3,i+7)
            sns.scatterplot(data=df[df['전화해지여부']==1], x=col_tuplt[0], y=col_tuplt[1], color='Orange')
            plt.ylim([-10, 70])
            
        plt.show()
            
    elif chart==2:
        df = pd.concat([X_, y_], axis=1)

        plt.figure(figsize=(14,14))
        target_list = ['주간통화시간', '주간통화횟수', '주간통화요금',
                       '저녁통화시간', '저녁통화횟수', '저녁통화요금',
                       '밤통화시간', '밤통화횟수', '밤통화요금']
        for i, col in enumerate(target_list):
            plt.subplot(3,3,i+1)
            sns.scatterplot(data=df, x='상담전화건수', y=col, hue='전화해지여부')
            plt.title(col)
        plt.tight_layout()
        plt.show()
        
        
    return X_, y_

# X_, y_ = trin_over_sampling(train, stype=1, chart=2)

## 학습

In [ ]:
# y = train['전화해지여부']
# X = train.drop(['전화해지여부'], axis=1)

y = df['전화해지여부']
X = df.drop(['전화해지여부'], axis=1)
# X_out = six_sigma(X)
X_scare = scaling_get_dummies(X, log=0)

param = {'criterion': ['gini'], 'max_depth': [50], 'n_estimators': [200]}

best_model = GridSearchCV(ExtraTreesClassifier(random_state=0), param, scoring='f1', cv=20)
best_model.fit(X_scare, y)
print(best_model.best_estimator_)
print(best_model.best_params_)
print(best_model.best_score_)

In [ ]:
def best_threshold(y_test, y_pro, rate=0.01, chart=0):
    accuracy, precision, recall, f1 = [],[],[],[]
    FPR, TPR = [],[]
    단위 = 0.01
    범위 = np.arange(0, 1, 단위)
    for 임계치 in 범위:
        y_ = Binarizer(threshold=임계치).fit_transform(y_pro).reshape(-1)
        TN, FP = 0, 0
        FN, TP = 0, 0
        for i in range(y_test.values.size):
            if y_test.values[i]==0 and y_[i]==0:
                TN = TN+1
            if y_test.values[i]==0 and y_[i]==1:
                FP = FP+1
            if y_test.values[i]==1 and y_[i]==0:
                FN = FN+1
            if y_test.values[i]==1 and y_[i]==1:
                TP = TP+1
        accuracy.append((TN+TP)/(TN+FN+FP+TP))
        precision.append(1 if FP+TP==0 else TP/(FP+TP))
        recall.append(TP/(FN+TP))
        f1.append(0 if (precision[-1]==0)|(recall[-1]==0) else 2*(precision[-1]*recall[-1])/(precision[-1]+recall[-1]))
        FPR.append(FP/(TN+FP))
        TPR.append(TP/(FN+TP))

    if chart==1:
        plt.figure(figsize=(14,6))
        plt.subplot(1,2,1)
        plt.plot(범위, recall, label='recall')
        plt.plot(범위, precision, label='precision')
        # plt.plot(범위, accuracy, label='accuracy')
        plt.plot(범위, f1, label='f1')
        plt.xlabel('threshold')
        plt.ylabel('score')
        plt.legend()
        plt.subplot(1,2,2)
        plt.plot(FPR[::-1], TPR[::-1])
        plt.plot([0,1], [0,1])
        plt.xlabel('FPR')
        plt.ylabel('TPR')
        plt.show()
        
    idx=np.argmax(f1)
    return np.round(idx*단위,3), f1[idx]

# best_threshold(pd.Series([0,0,0,1,1,1]), np.array([0.42, 0.2, 0.56, 0.49, 0.9, 0.99]).reshape(-1,1), rate=0.01, chart=1)

In [ ]:
scaling_get_dummies(test_df, log=0).shape

In [ ]:
plt.figure(figsize=(14,14))
sns.heatmap(train.corr(), annot=True, fmt='.2f', cmap='coolwarm')
plt.show()

In [ ]:
train = scaling_get_dummies(바보얌체(시간3분할(make_가입시즌(train))), sc=2)
y = train['전화해지여부']
X = train.drop(['전화해지여부'], axis=1)

threshold_list, f1_list = [],[]
res_model_list = []
for i, (train_idx, test_idx) in enumerate(StratifiedKFold(20).split(X, y)):
    X_train, y_train = X.loc[train_idx], y.loc[train_idx]
    X_test,  y_test  = X.loc[test_idx],  y.loc[test_idx]
    
    model = ExtraTreesClassifier(max_depth=50, n_estimators=200, random_state=11)
    model.fit(X_train, y_train)
    res_model_list.append(model)
    y_pro = model.predict_proba(X_test)    
    threshold, f1 = best_threshold(y_test, y_pro.T[1].reshape(-1,1))
    threshold_list.append(threshold)
    f1_list.append(f1)
    
res_df = pd.DataFrame()
res_df['threshold'] = threshold_list
res_df['f1_score'] = f1_list
res_df.sort_values(['f1_score'], ascending=False).head()

In [ ]:
s = pd.Series(res_model_list[10].feature_importances_, index=X.columns).sort_values()
sns.barplot(x=s.values, y=s.index)
plt.show()

## 피쳐재선정
<pre>

0 9.63%
1 9.59%
2 10.16%
3 16.44%
4 9.92%
5 24.79%
6 55.17%
7 53.85%
8 13.41%
9 83.33%
11 10.53%

In [ ]:
train['상담전화건수'] = train['상담전화건수'].replace([0,1,2,3,4,5,6,7,8,9,11], [0,0,0,0,0,1,1,1,1,1,1])

In [ ]:
scaling_get_dummies(바보얌체(시간3분할(make_가입시즌(train))), sc=2)

In [ ]:
test['상담전화건수'] = test['상담전화건수'].replace([0,1,2,3,4,5,6,7,8,9,11], [0,0,0,0,0,1,1,1,1,1,1])

In [ ]:
scaling_get_dummies(바보얌체(시간3분할(make_가입시즌(test))), sc=2).head(1)

In [ ]:
submit_arr = Binarizer(threshold=0.35).fit_transform(res_model_list[10].predict_proba(scaling_get_dummies(바보얌체(시간3분할(make_가입시즌(test))), sc=2))).T[1].reshape(-1,1)
submit_csv = pd.read_csv('./sample_submission.csv')
submit_csv['전화해지여부'] = submit_arr
submit_csv.to_csv('./submit(ku).csv', index=False)

In [ ]:
train[train['전화해지여부']==1]